# Homework 5

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
wage_raw = pd.read_csv('Wage.csv')

## 1. K-Means

1. Instead of just having an indicator for job class we add an indicator for each of the categorical variables. The categorical variables are:
* sex
* maritl
* race
* education
* region
* jobclass
* health
* health_ins

In [8]:
cat_var = ['sex', 'maritl', 'race', 'education', 'region', 'jobclass', 'health', 'health_ins']
wage = pd.get_dummies(wage_raw, columns=cat_var, drop_first=True)

In [9]:
wage

,year,age,logwage,wage,maritl_2. Married,maritl_3. Widowed,maritl_4. Divorced,maritl_5. Separated,race_2. Black,race_3. Asian,race_4. Other,education_2. HS Grad,education_3. Some College,education_4. College Grad,education_5. Advanced Degree,jobclass_2. Information,health_2. >=Very Good,health_ins_2. No
0,2006,18,4.318063,75.043154,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,2004,24,4.255273,70.476020,False,False,False,False,False,False,False,False,False,True,False,True,True,True
2,2003,45,4.875061,130.982177,True,False,False,False,False,False,False,False,True,False,False,False,False,False
3,2003,43,5.041393,154.685293,True,False,False,False,False,True,False,False,False,True,False,True,True,False
4,2005,50,4.318063,75.043154,False,False,True,False,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2008,44,5.041393,154.685293,True,False,False,False,False,False,False,False,True,False,False,False,True,False
2996,2007,30,4.602060,99.689464,True,False,False,False,False,False,False,True,False,False,False,False,True,True
2997,2005,27,4.193125,66.229408,True,False,False,False,True,False,False,False,False,False,False,False,False,True
2998,2005,27,4.477121,87.981033,False,False,False,False,False,False,False,False,True,False,False,False,True,False


2. Code

In [10]:
rng = np.random.default_rng()
train_idx = rng.binomial(1, 0.8, size = wage.shape[0]).astype(bool)
wage_train = wage[train_idx]
wage_test = wage[~train_idx]

3. We assume that everyone has completed middle school for a minimum of 9 years of schooling. If someone is a High School Grad that counts for 4 more years. If they have some college, we assume that's an associate degree. The cases where someone drops out at 1 year or 3 years average out to 2 years (most of the time). Some college should count for 2 years. College grad counts for 4 years. Advanced degreees can be masters or phds or professional degrees. Masters usually are 2 years, and Phds are usually 5-6, but often take much longer. We assume it adds a bit more years of schooling, say 4 years on average. 

4. Code

In [18]:
# make education years column
education_years = 9 + wage['education_2. HS Grad'] * 4 + wage['education_3. Some College'] * (2 + 4) + wage['education_4. College Grad'] * (4 + 4) + wage['education_5. Advanced Degree'] * (4 + 4 + 4)

# make kmeans data_frame
wage_kmeans = pd.DataFrame({'jobclass': wage['jobclass_2. Information'], 'age': wage['age'], 'education_years': education_years, 'logwage': wage['logwage']})

# standardize
wage_kmeans[['age', 'education_years', 'logwage']] = (wage_kmeans[['age', 'education_years', 'logwage']] - wage_kmeans[['age', 'education_years', 'logwage']].mean()) / wage_kmeans[['age', 'education_years', 'logwage']].std()

# split
wage_kmeans_train = wage_kmeans[train_idx]
wage_kmeans_test = wage_kmeans[~train_idx]

5. Code

In [22]:
import sklearn.cluster
kmeans_clf = sklearn.cluster.KMeans(n_clusters = 8)
kmeans_clf.fit(wage_kmeans_train)

KMeans()

6. Predict

In [ ]:
kmeans_preds = 
kmeans_clf.labels_

array([6, 3, 2, ..., 6, 6, 7], shape=(2430,), dtype=int32)